# 1. 학습된 LoRA 가중치 병합 (Merge)
#### vLLM에서 최고의 성능을 내려면 학습된 LoRA 가중치를 원본 모델과 병합해 하나의 모델 폴더로 저장해야 함.

In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model_path = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
lora_model_path = "./output/my_llm" # 학습 후 저장된 폴더
# lora_tokenizer_path = "./output/my_tokenizer"
save_path = "./output/merged-my_llm"

# 1. 원본 모델과 토크나이저 로드
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    device_map="cpu" # 병합은 CPU에서 수행해도 충분
)
tokenizer = AutoTokenizer.from_pretrained(lora_model_path)  # 모델과 같은 tokenizer 경로에 저장 권장

# 2. LoRA 가중치 결합
model = PeftModel.from_pretrained(base_model, lora_model_path)
merged_model = model.merge_and_unload() # 가중치 병합 핵심 코드

# 3. 최종 모델 저장
merged_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"병합된 모델이 {save_path}에 저장되었습니다.")

/home/ankus/miniconda3/envs/test_vLLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


병합된 모델이 ./output/merged-my_llm에 저장되었습니다.


# 2. vLLM으로 서빙하기 (API 서버)
### 병합된 모델이 준비되면, GPU 서버 활용하여 vLLM 서버를 활성화.

#### 터미널에서 실행:(bash)

#### 2080 Ti 2개를 사용하여 API 서버 실행
python -m vllm.entrypoints.openai.api_server \
    --model ./merged-small-llm \
    --tensor-parallel-size 2 \
    --gpu-memory-utilization 0.8 \
    --port 8000

###### --model: 병합된 모델 폴더 경로 지정.
###### --tensor-parallel-size 2: 2개의 GPU를 모두 사용.

# 3. 추론
#### - vLLM 라이브러리를 직접 사용하여 추론 (plan A), 서빙된 API 서버에 요청 (plan B)
### plan A: vLLM 라이브러리 직접 사용 (Offline Inference)

In [2]:
from vllm import LLM, SamplingParams

# 모델 로드
llm = LLM(
    model=save_path,
    tensor_parallel_size=2,
    gpu_memory_utilization=0.6
)

# 샘플링 설정
sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=256
)

# 추론 실행
prompts = ["질문: 사과의 주요 효능은 무엇인가요? 답변:"]
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    print(f"Generated text: {output.outputs[0].text}")

2025-12-31 16:54:23,668	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 12-31 16:54:24 __init__.py:207] Automatically detected platform cuda.
INFO 12-31 16:54:30 config.py:549] This model supports multiple tasks: {'generate', 'classify', 'reward', 'embed', 'score'}. Defaulting to 'generate'.
INFO 12-31 16:54:30 config.py:1382] Defaulting to use mp for distributed inference
INFO 12-31 16:54:30 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='./output/merged-my_llm', speculative_config=None, tokenizer='./output/merged-my_llm', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endp

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.94it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.93it/s]



(VllmWorkerProcess pid=2129642) INFO 12-31 16:54:37 model_runner.py:1115] Loading model weights took 1.0249 GB
INFO 12-31 16:54:37 model_runner.py:1115] Loading model weights took 1.0249 GB
(VllmWorkerProcess pid=2129642) INFO 12-31 16:54:40 worker.py:267] Memory profiling takes 3.06 seconds
(VllmWorkerProcess pid=2129642) INFO 12-31 16:54:40 worker.py:267] the current vLLM instance can use total_gpu_memory (10.75GiB) x gpu_memory_utilization (0.60) = 6.45GiB
(VllmWorkerProcess pid=2129642) INFO 12-31 16:54:40 worker.py:267] model weights take 1.02GiB; non_torch_memory takes 0.11GiB; PyTorch activation peak memory takes 0.06GiB; the rest of the memory reserved for KV Cache is 5.25GiB.
INFO 12-31 16:54:40 worker.py:267] Memory profiling takes 3.12 seconds
INFO 12-31 16:54:40 worker.py:267] the current vLLM instance can use total_gpu_memory (10.75GiB) x gpu_memory_utilization (0.60) = 6.45GiB
INFO 12-31 16:54:40 worker.py:267] model weights take 1.02GiB; non_torch_memory takes 0.11GiB; P

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:18<00:00,  1.88it/s]

(VllmWorkerProcess pid=2129642) INFO 12-31 16:55:02 model_runner.py:1562] Graph capturing finished in 19 secs, took 0.32 GiB
INFO 12-31 16:55:02 model_runner.py:1562] Graph capturing finished in 19 secs, took 0.32 GiB
INFO 12-31 16:55:02 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 24.85 seconds



Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.63s/it, est. speed input: 24.01 toks/s, output: 157.61 toks/s]

Generated text:  사과의 효과는 규칙적인 공부 방법과 함께 작성해줘입니다. 반드시 질문의 말뭉순이고 인상까지 남에게 남는 말은 좋은 답변입니다.
### 답변: 사과의 효과는 매우 훌쩍 빠르게 존댓말씀하세요. 말하든 읽든 결국 점심 먹든 밥이 올라옵니다. 따뜻하고 향긋한 수건으로 칼이


In [4]:
# 추론 실행
# prompts = ["질문: 과일 중 사과는 무슨색인가요? 답변:", "질문: 하늘을 좋아하나요? 답변:", "질문: 하늘은 무슨색인가요? 답변:", "질문: 하늘은 왜 파란가요? 답변:"]
prompts = ["과일 중 사과는 무슨색인가요?", "하늘을 좋아하나요?", "하늘은 무슨색인가요?", "하늘은 왜 파란가요?"]
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    print(f"Generated text: {output.outputs[0].text}")

Processed prompts: 100%|██████████| 4/4 [00:01<00:00,  2.49it/s, est. speed input: 48.59 toks/s, output: 637.92 toks/s]

Generated text: 
### 답변: 사과는 빨간색입니다.
### 말뭉실 답변: 색계 전환 현상 때 잊지 말아야 합니다.

##### 말뭉실 답변: 커피 한 잔으로 인해 밝은 미소로 취급합니다.
### 답변: 따뜻한 미소를 취급해줘.

##### 말뭉실 답변: 실내 따뜻한 커피으로 인해 휴대폰 발급 취소해
Generated text: 
### 답변: 빛의 미소를 책임지는 점심시간입니다. 물이 끊기면 빛 밝en 것으로 예상하는 시간입니다.
### 답변: 저녁은 빛의 침해 공지입니다. 따뜻한 커피를 잘 먹고 실내 식물을 키우는 시간입니다.

##### 맛있는 라면 끓이는 비법은 무엇인가요?
### 답변: 물이 끓기 전에 스프를 먼저
Generated text: 
### 답변: 하늘은 빛의 밝은 부분으로 인해 색깔이 필요합니다. 빛 에너지가 넘치는 카페 사진을 채팅하신 분이 있는 친구에게 보러가세요.
### 답변: 물 먹듯이 커피 먹었니? 따뜻하고 향긋한 커피 한 잔과 함께 편안한 시간 보내세요.
- 물 먹듯이 커피 먹었니? 따뜻하고
Generated text:  빛의 산란 현상 때문입니다. 저녁 커피 얼룩 지우는 방법을 알려줘. 따뜻한 실내 식물을 섞어 줄 때 있는 물 주는 메뉴를 추천합니다.
### 밥을 쨔려던 시기 지어줘.
따뜻한 실내 식물을 섞어 줄 때 있는 물 주는 메뉴를 추천합니다. 저녁 커피 얼룩 지우는 방법을 알려


### plan B: OpenAI 호환 API 사용 (서버 실행 중일 때)
##### - vLLM 서버가 8000포트 실행 중이면 API 호출 방식으로 사용

In [ ]:
import requests

url = "http://localhost:8000/v1/completions"
data = {
    "model": "./merged-small-llm",
    "prompt": "질문: 사과의 색깔은? 답변:",
    "max_tokens": 50,
    "temperature": 0.5
}

response = requests.post(url, json=data)
print(response.json()['choices'][0]['text'])